## March 03 2019


* Ideas that was floated in the meeting was pertinent and more relevant 
    * GO analysis on PP dataset as well with PD TAV sets 
    * PP dataset analysis with more analytic approach
    * Annotating the Distal element of PD interaction with TFs, Histone modifications and SNPs
    * Allele specific interaction in PD and PP
   
* Make a universal table of PD Table with all the functional element added
* EndoMT transitions in both Biscupid aortic valve and Tricuspid aortic valve

### ROADMAP FOR BAV-TAV DATA ANALYSIS

* Get all the PD with pvalue < 0.05 , length < = 10kb.

* Annontate these dataset with Differential interaction, Differentially expressed genes and Differential H3K27ac peaks.

* Populate this dataset with CTCF, (Other Cardiac specific)Transcription factor and DNAise HS from ENCODE, ROADMAP Epigenetics and FATHOM dataset from Aortic Endothelail cells 

* Find the Transcription factor binding sites (TFBS) in these regions

* Annotate with the phased single nucleotide polymorphism (SNP)  and CNVs from these samples in these regions

* Topological associated domains  (maybe)

## TASKS 

#### * Make a script that takes all of the interaction dataset (PD) ana adds column with RNAseq and CHipseq data to the existing table.

#### * Do diffchipseq and diffExpr in each conditions

#### * Add these results to the PD table


#### TASK 1 pseudo codes


* Read PD table in python 
* Make a dictionary based on gene 
* Read Expression files in same dataset


In [1]:
def int_dict_pd(file):
    """
        This function takes the interaction file of promoter distal interaction and returns dictionary of gene name as 
        the key and transcript_Id, distal chromosome position, as the list.
        
        file = interaction file from HiCap runs with RefSeqName as the gene_ID, TranscriptName as transcript_ID
    
    """
    results_interaction = {}
    with open (file, 'r') as int_files:
        next(int_files)
        
        for lines in int_files:
            line = lines.strip()
            fields = line.split("\t")
            if fields[11] == "-1":
                pass
            else:
                interaction_genes = fields[0]
                interaction_status_genes = results_interaction.get(interaction_genes,[])
                interaction_status_genes.append([fields[1], fields[8], fields[9], fields[10], fields[11]])
                results_interaction[interaction_genes] = interaction_status_genes
    return (results_interaction)

In [2]:
## Testing for the *.tab files output from stringtie

import os
import glob

os.chdir("/Volumes/Work_drive/prj/BAV_TAV/data/raw_internal/RNA_data")
stringtie_output = glob.glob('*.tab')

print(stringtie_output)
myorder = [2,3,0,1]
stringtie_output_sorted = [str(stringtie_output[i]) for i in myorder]
print(stringtie_output_sorted)

all_combnined = {}

for i in stringtie_output_sorted:
    with open (i, 'r') as tabfiles:
        next(tabfiles)
        
        expression_genes = {}
        
        for lines in tabfiles:
            line = lines.strip().split("\t")
            genes_ID = line[1]
            val_FPKM = expression_genes.get(genes_ID,[])
            val = line[-2]
            val_FPKM.append(val)
            expression_genes[genes_ID] = val_FPKM
                
        for keys, value in expression_genes.items():
            genes_ID = keys
            val = all_combnined.get(genes_ID,[])
            val.append(value)
            all_combnined[genes_ID] = val
            

['BAV2375_RNA.gene_abund.tab', 'BAV2424_RNA.gene_abund.tab', 'TAV2431_RNA.gene_abund.tab', 'TAV2515_RNA.gene_abund.tab']
['TAV2431_RNA.gene_abund.tab', 'TAV2515_RNA.gene_abund.tab', 'BAV2375_RNA.gene_abund.tab', 'BAV2424_RNA.gene_abund.tab']


In [3]:
list (all_combnined.keys()) [0:5]

['DDX11L1', 'WASH7P', 'MIR1302-2HG', 'RP11-34P13.7', 'RP11-34P13.8']

In [4]:
import os 
os.chdir ("/Volumes/Work_drive/prj/BAV_TAV/data/raw_internal/Interaction_march_corrected/")


fh = open("Probe_Distal_BAVTAV.filtered_expression.txt", "w")

with open ("BAVTAV.Proximities.Probe_Distal_SP4_p01_filtered.txt", 'r') as Int:
    for lines in Int:  
        if lines.startswith('RefSeqName'):
            line = lines.strip().split("\t")
            line.append ('width')
            line.append ('TAV2431_FPKMS')
            line.append ('TAV2515_FPKMS')
            line.append ('BAV2375_FPKMS')
            line.append ('BAV2424_FPKMS')
            #print (line)
            
            fh.write("\t".join(line))
            fh.write ("\n")
            continue
            powd 
        else:
            line = lines.strip().split("\t")
            wdth = int(line[10]) - int(line[9])
            dst = abs(int(line[11]))
            line.append(str(wdth))
            Expr_Genes = list (all_combnined.keys())
            
            
            if wdth < 5000 and dst < 5000000:
                
                if line[0] in Expr_Genes:
                    
                    for i  in all_combnined[line[0]]:
                        ind_exps = ",".join(i)
                        line.append (ind_exps )
                    fh.write("\t".join(line))
                    fh.write ("\n")
                
                else:
                    line.append("-1")
                    line.append("-1")
                    line.append("-1")
                    line.append("-1")
                    fh.write("\t".join(line))
                    fh.write ("\n")
                    
            
fh.close()

In [6]:
%%bash
pwd -P

 sed '1d'  Probe_Distal_BAVTAV.filtered_expression.txt | \
 awk -v OFS="\t" '{print $9,$10,$11,$3}' |sort -k 1,1 -k2,2n  |uniq > sorted_PD.bed 

/Volumes/Work_drive/prj/BAV_TAV/data/raw_internal/Interaction_march_corrected


In [7]:
import sys
import argparse
import os 
import pybedtools
import numpy as np


os.chdir ("/Volumes/Work_drive/prj/BAV_TAV/data/raw_internal/Interaction_march_corrected")

def bed2dict(bed):
    dict_intersect = {}
    for line in bed:
        genomic_coordinate = line[0],line[1],line[2]
        dict_intersect_keys = ",".join(genomic_coordinate)
        dict_intersect_values = dict_intersect.get(dict_intersect_keys,[])
        dict_intersect_values.append(line[7])
        dict_intersect[dict_intersect_keys] = dict_intersect_values
    return dict_intersect

def compAndret(list1,list2,list3):
    A = ",".join(list1)
    if A in list2:
        test = list3[A]
        Mark = (" ".join(np.unique(test)))
    else:
        Mark = str(0)
    return(Mark)

file1 = "sorted_PD.bed"
file2 = "TAV_Replicates.Bed"
file3 = "BAV_Replicates.Bed"


interact_bed = pybedtools.BedTool(file1)
TAV_his_bed = pybedtools.BedTool(file2)
BAV_his_bed = pybedtools.BedTool(file3)

TAV_intersect = interact_bed.intersect(TAV_his_bed, wo = True)
BAV_intersect = interact_bed.intersect(BAV_his_bed, wo = True)

TAV_dict = bed2dict(TAV_intersect)
BAV_dict = bed2dict(BAV_intersect)

keys_TAV_dict = list (TAV_dict.keys())
keys_BAV_dict = list (BAV_dict.keys())

interaction_file = "Probe_Distal_BAVTAV.filtered_expression.txt"

filename_out = "PD_Exprs_H3K27ac.txt"
fh = open(filename_out, "w")

with open (interaction_file, 'r') as int_files:
    for lines in int_files:
        if lines.startswith('RefSeqName'):
            line = lines.strip().split("\t")
            line.append ('TAV_H3K27ac_overlap_width')
            line.append ('BAV_H3K27ac_overlap_width')
            fh.write("\t".join(line))
            fh.write ("\n")
        
        else:
            A = lines.strip().split("\t")
            coordinates =[A[8],A[9],A[10]]

            TAV_His = compAndret(coordinates, keys_TAV_dict, TAV_dict)
            BAV_His = compAndret(coordinates, keys_BAV_dict, BAV_dict)
  
            res = lines.strip()+"\t"+TAV_His+"\t"+BAV_His
            fh.write (res)
            fh.write ("\n")
            
fh.close()

In [8]:
%%bash

rm -r Probe_Distal_BAVTAV.filtered_expression.txt